# Align Bionet with Wikidata 


- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*



## Get artists from wikidata

In [1]:
from random import randint
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON
from time import sleep


# Get artists from intavia bionet

In [3]:
# Get all artists from Intavia bionet 

myquery = "Mondriaan"

sparql = SPARQLWrapper("http://130.37.53.36:6789/sparql")
sparql.setQuery("""
SELECT ?person (GROUP_CONCAT(?namelabel;SEPARATOR=";") AS ?names) (GROUP_CONCAT(?tslab;SEPARATOR=";") AS ?tslabs) WHERE {
    ?s bioc:has_occupation bgn:Occupation-schilder.
    ?s idm:person_proxy_for ?person.
    ?s crm:P1_is_identified_by ?name .
    ?name rdf:type crm:E33_E41_Linguistic_Appellation . 
    ?name rdfs:label ?namelabel . 
    OPTIONAL {
     ?bev crm:P98_brought_into_life ?s .
     ?bev crm:P4_has_time-span ?ts.
     ?ts rdfs:label ?tslab}
}
GROUP BY ?person

""")
sparql.setReturnFormat(JSON)
bionetresults = sparql.query().convert()
for result in bionetresults["results"]["bindings"]:
    print(result["person"]["value"] + '\t' + result["names"]["value"] + '\t' + result["tslabs"]["value"])  
    print()

URLError: <urlopen error [Errno 61] Connection refused>

In [4]:
# For a string, return a set of wikidata ids of things that are artists

def lookupnameInWikidata(name):
    resultlist = []
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
        SELECT DISTINCT * WHERE { 
        SERVICE wikibase:mwapi 
              { bd:serviceParam wikibase:api "EntitySearch" . 
               bd:serviceParam wikibase:endpoint "www.wikidata.org" . 
               bd:serviceParam mwapi:search " """ +name+ """". 
               bd:serviceParam mwapi:language "en" . 
               ?item wikibase:apiOutputItem mwapi:item . 
               ?num wikibase:apiOrdinal true . } 
        ?item wdt:P106/wdt:P279* wd:Q483501 .
        } ORDER BY ?item
    """)
    sparql.setReturnFormat(JSON)
    wikidataresults = sparql.query().convert()
    for result in wikidataresults["results"]["bindings"]:
        resultlist.append(result["item"]["value"])
    return resultlist

In [5]:
# Main lookup loop

offset=102
i=0
for artist in bionetresults["results"]["bindings"]:
    if i >= offset:
        onename = artist["names"]["value"]
        namesplit = onename.split(';')
        if len(namesplit) > 0: 
            for namevar in namesplit:
                if ',' in namevar:
                    namevarlist = namevar.split(',')
                    newnamevar = namevarlist[1]+namevarlist[0]
                else: 
                    newnamevar = namevar
                print("--" + newnamevar + "--")
                print(str(i) + '\t' + artist["person"]["value"] + '\t' + artist["names"]["value"] + '\t' + artist["tslabs"]["value"] + '\t' + newnamevar + '\t' + str(lookupnameInWikidata(newnamevar)))
                sleep(randint(5,8))
    i=i+1


NameError: name 'bionetresults' is not defined

In [6]:
print(len(bionetresults["results"]["bindings"]))


NameError: name 'bionetresults' is not defined

# Get artworks and some metadata from wikidata

In [7]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE

endpoint_url = "https://query.wikidata.org/sparql"

query = """CONSTRUCT {?artwork wdt:P170 ?artist . 
           ?artwork wdt:P18 ?image .
           ?artwork wdt:P1476 ?title . 
          ?artwork wdt:P571 ?date .}
WHERE
{
       ?artist wdt:P651 ?bartist .
       ?artwork wdt:P170 ?artist . 
  
OPTIONAL { ?artwork wdt:P18 ?image .
       ?artwork wdt:P1476 ?title . 
       ?artwork wdt:P571 ?date . }
}

LIMIT 100

"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(TURTLE)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

print(results.serialize())

@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

wd:Q1025704 wdt:P1476 "Terrasse du café le soir"@fr ;
    wdt:P170 wd:Q5582 ;
    wdt:P18 <http://commons.wikimedia.org/wiki/Special:FilePath/Vincent%20van%20Gogh%20%281853-1890%29%20Caf%C3%A9terras%20bij%20nacht%20%28place%20du%20Forum%29%20Kr%C3%B6ller-M%C3%BCller%20Museum%20Otterlo%2023-8-2016%2013-35-40.JPG> ;
    wdt:P571 "1888-09-01T00:00:00+00:00"^^xsd:dateTime .

wd:Q1057817 wdt:P1476 "Woman with a Pink"@en ;
    wdt:P170 wd:Q5598 ;
    wdt:P18 <http://commons.wikimedia.org/wiki/Special:FilePath/Woman%20with%20a%20Pink%20MET%20DP145910.jpg> ;
    wdt:P571 "1660-01-01T00:00:00+00:00"^^xsd:dateTime .

wd:Q1134060 wdt:P1476 "Susanna en de oudsten"@nl ;
    wdt:P170 wd:Q5599 ;
    wdt:P18 <http://commons.wikimedia.org/wiki/Special:FilePath/Rubens%20Susanna.jpg> ;
    wdt:P571 "1607-01-01T00:00:00+00:00"^^xsd:dateTime .

wd:Q113425

/Users/gsugimoto/opt/anaconda3/lib/python3.9/site-packages/SPARQLWrapper/Wrapper.py:1315: RuntimeWarning: Format requested was TURTLE, but RDF/XML (application/rdf+xml;charset=utf-8) has been returned by the endpoint
  warnings.warn(message % (requested.upper(), format_name, mime), RuntimeWarning)


In [8]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace, parse #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's

df=pd.read_csv('./query2.csv',sep=",",quotechar='"')
df # uncomment to check for contents

ImportError: cannot import name 'parse' from 'rdflib' (/Users/gsugimoto/opt/anaconda3/lib/python3.9/site-packages/rdflib/__init__.py)

In [9]:

g = Graph()

for index, row in df.iterrows():
    print(row['predicate'])
    if row['predicate']== 'http://www.wikidata.org/prop/direct/P571':
        g.add((URIRef(row['subject']), URIRef(row['predicate']), Literal(row['object'], datatype=XSD.date) ))
    elif row['predicate']== 'http://www.wikidata.org/prop/direct/P1476':
        g.add((URIRef(row['subject']), URIRef(row['predicate']), Literal(row['object']) ))
    else:
        g.add((URIRef(row['subject']), URIRef(row['predicate']), URIRef(row['object'])))


NameError: name 'df' is not defined

In [10]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
#print(g.serialize(format='turtle').decode('UTF-8'))
#g.serialize('mycsv2rdf.ttl',format='turtle')
g1 = Graph()
#g1.parse('wikidata2bionet.ttl', format="nt" ) #results of linking wd to bgn
#g1.parse('mycsv2rdf.ttl', format="turtle" )
#g1.serialize('bgn2wd.ttl',format='turtle')

p651 = URIRef("http://www.wikidata.org/prop/direct/P651") 
g2 = Graph()
for wda, bgna in g1.subject_objects(p651): 
    g2.add((URIRef(bgna),RDF.type, print(wda + bgna)


SyntaxError: unexpected EOF while parsing (3223606558.py, line 12)